<a href="https://colab.research.google.com/github/renatoerss/vch-srna-pred/blob/main/vch_srna_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!apt-get -q update
!apt-get -q install -y bedtools
!pip install -q pybigwig pandas numpy scipy pybedtools gffutils openpyxl

import shutil, importlib
for pip_name, mod in [("pybigwig","pyBigWig"),("pandas","pandas"),("numpy","numpy"),
                      ("scipy","scipy"),("pybedtools","pybedtools"),("gffutils","gffutils"),("openpyxl","openpyxl")]:
    try:
        m = importlib.import_module(mod)
        print("OK", pip_name, getattr(m,"__version__", ""))
    except Exception as e:
        print("ERRO", pip_name, e)
print("bedtools:", shutil.which("bedtools"))


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,425 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,275 kB

## 1. Configuration and Input Files

This section defines all input files and adjustable parameters for the analysis.  
Update the file paths (`/path/to/...`) to match your own dataset locations.

**Required inputs**
- Four strand-specific RNA-seq bigWig files:
  - `CRP (+/−)` and `K52Q (+/−)`
- Two genome annotations (GFF3 or converted XLSX), one per chromosome  
  *(e.g., `NC_002505` and `NC_002506`)*
- Two TSS maps in BED-like format (`TSS_plus.txt`, `TSS_minus.txt`)

**Output structure**
All results will be written automatically to the directory specified by `OUT_DIR`,  
with the following subfolders created for organization:




**Adjustable parameters**
- `SMOOTH_WIN` – smoothing window size for bigWig traces  
- `MIN_WIDTH_BP`, `MAX_WIDTH_BP` – min/max allowed peak width (bp)  
- `MIN_PROM` – minimum prominence threshold  
- `MIN_AREA` – minimum integrated area threshold  
- `FOLD_MIN` – minimum fold-difference between K52Q and CRP  
- `PROFILE_STEP` – sampling interval for quick QC profiling  

Before running the analysis:
1. Confirm that all paths exist by running the `_check()` helper.  
2. Adjust thresholds based on signal scale (median, p95/p99) from the QC step.  
3. Keep all edits confined to this configuration cell for reproducibility.


In [5]:
# ===============================================================
# 1) CONFIGURATION — EDIT THIS SECTION ONLY
# ===============================================================

from pathlib import Path
import os, json, re

# ====== INPUT FILES (EDIT THESE PATHS) ======

# bigWig tracks for strand-specific RNA-seq data
BIGWIGS = {
    "CRP": {
        "+": "/path/to/CRP_plus_stranded.bw",   # e.g., Drive or local path
        "-": "/path/to/CRP_minus_stranded.bw",
    },
    "K52Q": {
        "+": "/path/to/K52Q_plus_stranded.bw",
        "-": "/path/to/K52Q_minus_stranded.bw",
    }
}

# GFF3 or converted XLSX genome annotation files
# one per chromosome (example for Vibrio cholerae)
GFF3 = {"GFF3": {
    "NC_002505": "/path/to/NC_002505_GFF3converted.xlsx",
    "NC_002506": "/path/to/NC_002506_GFF3converted.xlsx",
}}

# TSS maps (BED-like format; strand-aware)
TSS_PLUS  = "/path/to/TSS_plus.txt"
TSS_MINUS = "/path/to/TSS_minus.txt"

# Output directory (all results will be stored here)
OUT_DIR = "/path/to/output/srna_scan_YYYYMMDD"

# ====== ANALYSIS PARAMETERS (ADJUST AS NEEDED) ======

SMOOTH_WIN   = 31            # smoothing window size (odd integer)
MIN_WIDTH_BP = 50            # minimum peak width (bp)
MAX_WIDTH_BP = 350           # maximum peak width (bp)
MIN_PROM     = 2e4           # minimum peak prominence (adjust per p95/p99)
MIN_AREA     = 3e5           # minimum integrated area of the peak
FOLD_MIN     = 5.0           # minimum fold-change (K52Q vs CRP)
PROFILE_STEP = 10            # sampling step for QC profiling
# ===============================================================


# ---------------------------------------------------------------
# Quick existence check utility
# ---------------------------------------------------------------
def _check(p):
    """Check if a file or folder exists and print result."""
    print(Path(p).exists(), p)

# Example usage (uncomment lines below to verify file paths)
# for k in ("+","-"):
#     _check(BIGWIGS["CRP"][k]); _check(BIGWIGS["K52Q"][k])
# _check(GFF3["GFF3"]["NC_002505"]); _check(GFF3["GFF3"]["NC_002506"])
# _check(TSS_PLUS); _check(TSS_MINUS)


# ---------------------------------------------------------------
# Prepare output directories
# ---------------------------------------------------------------
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)
(Path(OUT_DIR) / "mask").mkdir(exist_ok=True)
(Path(OUT_DIR) / "qc").mkdir(exist_ok=True)
(Path(OUT_DIR) / "peaks").mkdir(exist_ok=True)


In [7]:
import os, math, csv, warnings, itertools
from pathlib import Path
import numpy as np
import pandas as pd
import pyBigWig
from scipy.signal import find_peaks

warnings.filterwarnings("ignore", category=UserWarning)

# ---------------- Utilities ----------------
def canonical(chrom: str) -> str:
    """Remove version suffixes (e.g., '.1') for normalization."""
    return chrom.split('.')[0]

def read_bigwig(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"BigWig does not exist: {path}")
    return pyBigWig.open(path)

def chrom_sizes_from_bw(bw):
    """Return dict {canonical: (real_name, length)}."""
    out = {}
    for name, length in bw.chroms().items():
        can = canonical(name)
        if can not in out:
            out[can] = (name, length)
        else:
            # size consistency check
            if out[can][1] != length:
                print(f"[WARNING] Different sizes for {can}: {out[can][1]} vs {length}")
    return out

def norm_chrom_name_for_bw(bw, can_name):
    """Map canonical name to the actual chromosome name used in the BigWig."""
    for n, l in bw.chroms().items():
        if canonical(n) == can_name:
            return n
    return None

def read_gff_any(path):
    """
    Read GFF3 (.gff, .gff3, .gff.gz) OR converted XLSX.
    Returns DataFrame with columns: chrom, type, start, end, strand
    """
    path = str(path)
    if path.endswith((".gff", ".gff3", ".gff.gz")):
        rows = []
        opener = open
        if path.endswith(".gz"):
            import gzip
            opener = gzip.open
        with opener(path, "rt") as fh:
            for ln in fh:
                if not ln or ln.startswith("#"):
                    continue
                parts = ln.rstrip("\n").split("\t")
                if len(parts) < 5:
                    continue
                seqid, source, typ, start, end = parts[0], parts[1], parts[2], int(parts[3]), int(parts[4])
                strand = parts[6] if len(parts) > 6 else "."
                rows.append((canonical(seqid), typ, start, end, strand))
        df = pd.DataFrame(rows, columns=["chrom","type","start","end","strand"])
        return df
    elif path.endswith((".xlsx",".xls")):
        x = pd.read_excel(path, engine="openpyxl")
        # normalize column names
        cols = {c.lower(): c for c in x.columns}
        # try synonyms
        chrom_col = next((cols[k] for k in ["seqid","chrom","chromosome","contig","chr"] if k in cols), None)
        type_col  = next((cols[k] for k in ["type","feature","feat_type"] if k in cols), None)
        start_col = next((cols[k] for k in ["start","start_bp","start_pos","begin"] if k in cols), None)
        end_col   = next((cols[k] for k in ["end","end_bp","end_pos","stop"] if k in cols), None)
        strand_col= next((cols[k] for k in ["strand","str"] if k in cols), None)

        if not all([chrom_col, type_col, start_col, end_col]):
            raise ValueError(f"XLSX is missing required columns (chrom/type/start/end): {path}")

        df = pd.DataFrame({
            "chrom":  x[chrom_col].astype(str).map(canonical),
            "type":   x[type_col].astype(str),
            "start":  x[start_col].astype(int),
            "end":    x[end_col].astype(int),
            "strand": x[strand_col].astype(str) if strand_col else ".",
        })
        return df
    else:
        raise ValueError(f"Unsupported GFF format: {path}")

def merge_intervals(intervals):
    """intervals: list[(start,end)] -> merged, sorted, non-overlapping intervals."""
    if not intervals:
        return []
    arr = sorted(intervals, key=lambda x: (x[0], x[1]))
    merged = [arr[0]]
    for s,e in arr[1:]:
        ls, le = merged[-1]
        if s <= le + 1:
            merged[-1] = (ls, max(le, e))
        else:
            merged.append((s,e))
    return merged

def complement_intervals(covered, chrom_len):
    """covered intervals [(s,e)] -> complement within [1, chrom_len] (1-based)."""
    inter = []
    prev_end = 0
    for s,e in covered:
        if s > prev_end + 1:
            inter.append((prev_end+1, s-1))
        prev_end = e
    if prev_end < chrom_len:
        inter.append((prev_end+1, chrom_len))
    return inter

def write_bed(path, chrom, intervals):
    with open(path, "a") as fh:
        for s,e in intervals:
            fh.write(f"{chrom}\t{s-1}\t{e}\n")  # BED 0-based half-open

def values_from_bw(bw, chrom_real, start_1, end_1):
    """Return np.array of 1-bp values; replace NaN with 0."""
    # BigWig uses 0-based; here we use 1-based coordinates
    arr = np.array(bw.values(chrom_real, start_1-1, end_1), dtype=float)
    arr[np.isnan(arr)] = 0.0
    return arr

def moving_average(x, win):
    if win <= 1:
        return x
    # enforce odd window
    if win % 2 == 0:
        win += 1
    k = np.ones(win, dtype=float) / win
    return np.convolve(x, k, mode="same")

# ------------- Basic QC -------------
def qc_open_and_common_chroms(BIGWIGS):
    bws = {cond: {st: read_bigwig(path) for st, path in d.items()} for cond, d in BIGWIGS.items()}
    # collect sizes from one reference (CRP '+')
    ref_bw = bws["CRP"]["+"]
    sizes_ref = chrom_sizes_from_bw(ref_bw)  # {can: (real, len)}
    # find common chromosomes across all BigWigs
    commons = set(sizes_ref.keys())
    for cond, d in bws.items():
        for st, bw in d.items():
            sizes = chrom_sizes_from_bw(bw).keys()
            commons = commons.intersection(set(sizes))
    commons = sorted(list(commons))
    print("[QC] Common chromosomes (canonical):", commons)
    return bws, commons, sizes_ref

def read_mask_intergenic(GFF3_paths_by_can, commons, sizes_map):
    # Read GFF and filter to CDS/rRNA/tRNA only
    gff_all = []
    for can, path in GFF3_paths_by_can.items():
        if can not in commons:
            print(f"[NOTICE] {can} is not in the common set; skipping.")
            continue
        df = read_gff_any(path)
        df = df[df["chrom"] == can].copy()
        df = df[df["type"].isin(["CDS","rRNA","tRNA"])]
        gff_all.append(df)
    if not gff_all:
        raise RuntimeError("No valid GFF after filtering.")
    gff = pd.concat(gff_all, ignore_index=True)
    # merge per chromosome
    covered_by_chrom = {}
    for can in sorted(set(gff["chrom"])):
        sub = gff[gff["chrom"]==can][["start","end"]].to_numpy()
        merged = merge_intervals([(int(s),int(e)) for s,e in sub])
        covered_by_chrom[can] = merged
    # intergenic complement
    interg_by_chrom = {}
    for can in covered_by_chrom:
        chrom_len = sizes_map[can][1]
        interg_by_chrom[can] = complement_intervals(covered_by_chrom[can], chrom_len)
    return covered_by_chrom, interg_by_chrom

def report_mask_intergenic(covered_by_chrom, interg_by_chrom, sizes_map):
    rows = []
    for can in sorted(interg_by_chrom.keys()):
        cov_bp = sum(e - s + 1 for s,e in covered_by_chrom[can])
        ig_bp  = sum(e - s + 1 for s,e in interg_by_chrom[can])
        total  = sizes_map[can][1]
        rows.append((can, total, cov_bp, ig_bp, 100*cov_bp/total, 100*ig_bp/total, len(interg_by_chrom[can])))
    df = pd.DataFrame(rows, columns=["chrom","len_bp","masked_bp","intergenic_bp","masked_%","intergenic_%","n_intergenic"])
    return df

# ------------- Signal profiling -------------
def profile_track_over_intergenics(bw, sizes_map, interg_by_chrom, step=10, smooth_win=31):
    vals = []
    for can, intervals in interg_by_chrom.items():
        chrom_real = sizes_map[can][0]
        for s,e in intervals:
            if e - s + 1 < 3:
                continue
            # sample every 'step' bp
            xs = values_from_bw(bw, chrom_real, s, e)
            if step > 1:
                xs = xs[::step]
            if smooth_win and smooth_win > 1:
                xs = moving_average(xs, smooth_win)
            vals.append(xs)
    if not vals:
        return {"count_points": 0}
    v = np.concatenate(vals)
    v = v[np.isfinite(v)]
    if v.size == 0:
        return {"count_points": 0}
    med = float(np.median(v))
    mad = float(np.median(np.abs(v - med)))
    p95 = float(np.percentile(v, 95))
    p99 = float(np.percentile(v, 99))
    frac_pos = float(np.mean(v > 0))
    return {"count_points": int(v.size), "median": med, "MAD": mad, "p95": p95, "p99": p99, "frac_gt0": frac_pos}

# ------------- TSS (nearest) -------------
def read_tss_file(path):
    """Read BED-like file (ignores 'track'). Return DF: chrom, pos(1-based), name, strand."""
    rows = []
    with open(path, "r") as fh:
        for ln in fh:
            if not ln.strip() or ln.startswith("track") or ln.startswith("#"):
                continue
            parts = re.split(r"\s+|\t+", ln.strip())
            if len(parts) < 3:
                continue
            chrom = canonical(parts[0])
            start0 = int(parts[1])  # BED start (0-based)
            end1   = int(parts[2])  # BED end   (1-based)
            name   = parts[3] if len(parts) >= 4 else "."
            score  = parts[4] if len(parts) >= 5 else "."
            strand = parts[5] if len(parts) >= 6 else "."
            # TSS position: use start for '+', end for '-' (BED heuristic)
            pos1 = start0 + 1 if strand == "+" else end1
            rows.append((chrom, pos1, name, strand))
    df = pd.DataFrame(rows, columns=["chrom","pos1","name","strand"])
    return df

def build_tss_index(df):
    """Index by chromosome: sorted arrays of positions and names."""
    idx = {}
    for can, sub in df.groupby("chrom"):
        o = sub.sort_values("pos1")
        idx[can] = (o["pos1"].to_numpy(), o["name"].astype(str).to_numpy(), o["strand"].astype(str).to_numpy())
    return idx

def nearest_pos_name(arr_pos, arr_name, q):
    """Return (pos, name, dist) nearest to query q in arr_pos."""
    import bisect
    i = bisect.bisect_left(arr_pos, q)
    cand = []
    if i < len(arr_pos): cand.append((abs(arr_pos[i]-q), arr_pos[i], arr_name[i]))
    if i > 0:            cand.append((abs(arr_pos[i-1]-q), arr_pos[i-1], arr_name[i-1]))
    if not cand: return (None, None, None)
    d, p, nm = min(cand, key=lambda z: z[0])
    return (int(p), str(nm), int(d))

# ------------- Peak detection -------------
def detect_srnas_in_intergenics(bw_crp, bw_k52q, sizes_map, interg_by_chrom,
                                smooth_win=31, min_width=50, max_width=350, min_prom=2e4,
                                min_area=3e5, fold_min=5.0):
    """
    For each intergenic region: use max(smooth(K52Q), smooth(CRP)) to locate peaks,
    integrate area in CRP and K52Q over the same window, then apply filters.
    Returns (ALL, HITS) as lists of dicts.
    """
    ALL = []
    HITS = []
    for can, intervals in interg_by_chrom.items():
        real = sizes_map[can][0]
        for s,e in intervals:
            L = e - s + 1
            if L < min_width:
                continue
            # extract
            crp = values_from_bw(bw_crp, real, s, e)
            k52 = values_from_bw(bw_k52q, real, s, e)
            # smoothing
            crp_s = moving_average(crp, smooth_win)
            k52_s = moving_average(k52, smooth_win)
            # combined track
            comb = np.maximum(crp_s, k52_s)
            # find_peaks: widths in number of samples (1 sample = 1 bp)
            peaks, props = find_peaks(comb, prominence=min_prom, width=(min_width, max_width))
            if peaks.size == 0:
                continue
            # convert widths to integer bounds
            left_ips  = props.get("left_ips", np.array([], dtype=float))
            right_ips = props.get("right_ips", np.array([], dtype=float))
            prominences = props.get("prominences", np.zeros_like(peaks, dtype=float))
            for i, pk in enumerate(peaks):
                ls = int(max(0, math.floor(left_ips[i]))) if i < len(left_ips) else max(0, pk - min_width//2)
                rs = int(min(L-1, math.ceil(right_ips[i]))) if i < len(right_ips) else min(L-1, pk + min_width//2)
                # area (raw sum, not smoothed)
                area_crp = float(np.sum(crp[ls:rs+1]))
                area_k52 = float(np.sum(k52[ls:rs+1]))
                width_bp = int(rs - ls + 1)
                prom     = float(prominences[i]) if i < len(prominences) else float(comb[pk])
                # collect ALL
                rec = {
                    "chrom_can": can,
                    "start1":    int(s + ls),
                    "end1":      int(s + rs),
                    "peak_pos1": int(s + pk),
                    "width_bp":  width_bp,
                    "prominence": prom,
                    "area_CRP":  area_crp,
                    "area_K52Q": area_k52,
                }
                ALL.append(rec)
                # filters (EXAMPLE thresholds)
                mag = max(area_crp, area_k52)
                sm  = min(area_crp, area_k52) + 1e-9
                fold = mag / sm
                if (mag >= min_area) and (fold >= fold_min):
                    who = "K52Q>CRP" if area_k52 >= area_crp else "CRP>K52Q"
                    hit = rec | {"fold": float(fold), "who": who}
                    HITS.append(hit)
    return ALL, HITS

def save_bed_and_tsv(outdir, label, strand, all_list, hits_list):
    bed_all  = Path(outdir)/"peaks"/f"{label}_ALL.{strand}.bed"
    bed_hits = Path(outdir)/"peaks"/f"{label}_HITS.{strand}.bed"
    tsv      = Path(outdir)/"peaks"/f"{label}_RESULTS.{strand}.tsv"
    # save BEDs
    with open(bed_all, "w") as fa:
        for r in all_list:
            fa.write(f"{r['chrom_can']}\t{r['start1']-1}\t{r['end1']}\n")
    with open(bed_hits, "w") as fh:
        for r in hits_list:
            fh.write(f"{r['chrom_can']}\t{r['start1']-1}\t{r['end1']}\t{r.get('who','.')}\t0\t.\n")
    # TSV
    pd.DataFrame(hits_list).to_csv(tsv, sep="\t", index=False)
    return bed_all, bed_hits, tsv

# ------------- MAIN -------------
def main():
    print("== 2) QC: open BigWigs and determine common chromosomes ==")
    bws, commons, sizes_ref = qc_open_and_common_chroms(BIGWIGS)
    if not commons:
        raise RuntimeError("No common chromosomes among the BigWigs.")

    # 1) path checks
    print("Checking inputs...")
    for cond in BIGWIGS:
        for st in ("+","-"):
            p = BIGWIGS[cond][st]
            print(os.path.exists(p), p)
    for can, p in GFF3["GFF3"].items():
        print(os.path.exists(p), p)
    print(os.path.exists(TSS_PLUS), TSS_PLUS)
    print(os.path.exists(TSS_MINUS), TSS_MINUS)

    print("\n== 3) Healthy mask (CDS, rRNA, tRNA) and intergenics ==")
    covered_by_chrom, interg_by_chrom = read_mask_intergenic(GFF3["GFF3"], commons, sizes_ref)
    rep = report_mask_intergenic(covered_by_chrom, interg_by_chrom, sizes_ref)
    rep_path = Path(OUT_DIR)/"mask"/"mask_intergenic_report.tsv"
    rep.to_csv(rep_path, sep="\t", index=False)
    print(rep)

    # save BEDs
    mask_bed = Path(OUT_DIR)/"mask"/"mask_CDS_rRNA_tRNA.bed"
    inter_bed= Path(OUT_DIR)/"mask"/"intergenic_from_CDS_rRNA_tRNA.bed"
    # remove if they already exist
    for p in (mask_bed, inter_bed):
        if p.exists(): p.unlink()
    for can in sorted(covered_by_chrom.keys()):
        write_bed(mask_bed, can, covered_by_chrom[can])
        write_bed(inter_bed, can, interg_by_chrom[can])
    print("BEDs saved:", mask_bed, inter_bed)

    print("\n== 4) Signal profile (short smoothing) ==")
    prof = []
    for st in ("+","-"):
        pr_crp  = profile_track_over_intergenics(bws["CRP"][st], sizes_ref, interg_by_chrom, step=PROFILE_STEP, smooth_win=31)
        pr_k52  = profile_track_over_intergenics(bws["K52Q"][st], sizes_ref, interg_by_chrom, step=PROFILE_STEP, smooth_win=31)
        pr_crp["track"] = f"CRP_{st}"
        pr_k52["track"] = f"K52Q_{st}"
        prof.extend([pr_crp, pr_k52])
    df_prof = pd.DataFrame(prof)
    prof_path = Path(OUT_DIR)/"qc"/"profiles.tsv"
    df_prof.to_csv(prof_path, sep="\t", index=False)
    print(df_prof)

    print("\n== 5) sRNA-like peak detection per strand (no fixed windows) ==")
    results = []
    for st in ("+","-"):
        ALL, HITS = detect_srnas_in_intergenics(
            bws["CRP"][st], bws["K52Q"][st], sizes_ref, interg_by_chrom,
            smooth_win=SMOOTH_WIN, min_width=MIN_WIDTH_BP, max_width=MAX_WIDTH_BP,
            min_prom=MIN_PROM, min_area=MIN_AREA, fold_min=FOLD_MIN,
        )
        bed_all, bed_hits, tsv = save_bed_and_tsv(OUT_DIR, "SRNA", st, ALL, HITS)
        print(f"[{st}] candidates (ALL): {len(ALL)}  |  HITS (filtered): {len(HITS)}")
        results.append((st, len(ALL), len(HITS), bed_all, bed_hits, tsv))

    print("\n== 6) Nearest TSS (strand-aware) ==")
    tss_plus = read_tss_file(TSS_PLUS)
    tss_minus= read_tss_file(TSS_MINUS)
    idx_plus = build_tss_index(tss_plus)
    idx_minus= build_tss_index(tss_minus)

    # attach nearest TSS to each RESULT.tsv
    for st, n_all, n_hits, bed_all, bed_hits, tsv in results:
        df = pd.read_csv(tsv, sep="\t")
        if df.empty:
            print(f"[{st}] No HITS; skipping nearest TSS.")
            continue
        pos_arr = []
        name_arr= []
        dist_arr= []
        for _, r in df.iterrows():
            can = r["chrom_can"]
            q   = int((r["start1"] + r["end1"])//2)  # peak center
            if st == "+" and can in idx_plus:
                pos, nm, dist = nearest_pos_name(idx_plus[can][0], idx_plus[can][1], q)
            elif st == "-" and can in idx_minus:
                pos, nm, dist = nearest_pos_name(idx_minus[can][0], idx_minus[can][1], q)
            else:
                pos, nm, dist = (None, None, None)
            pos_arr.append(pos); name_arr.append(nm); dist_arr.append(dist)
        df["nearest_tss_pos1"] = pos_arr
        df["nearest_tss_name"] = name_arr
        df["dist_tss_bp"]      = dist_arr
        out_tsv = Path(OUT_DIR)/"peaks"/f"SRNA_RESULTS.{st}.nearest.tsv"
        df.to_csv(out_tsv, sep="\t", index=False)
        print(f"[{st}] RESULT + nearest TSS:", out_tsv)

    print("\n== 8) Visualization ==")
    print("Use igv.org/app and load: BigWigs (+/−); intergenic and HITS BEDs; GFF3 (if available as .gff3).")
    print("To share from Drive, create read-only public links.")

    print("\n== 9) Quick diagnostics ==")
    rep = pd.read_csv(Path(OUT_DIR)/"mask"/"mask_intergenic_report.tsv", sep="\t")
    if (rep["n_intergenic"]==0).any():
        print("intergenics = 0 → GFF3 and BigWigs do not match (names/versions). Check '.1' and chromosome names.")
    # check HITS
    totals = sum(x[2] for x in results)
    alls   = sum(x[1] for x in results)
    if alls == 0:
        print("ALL=0 → thresholds too high (MIN_PROM/MIN_WIDTH/SMOOTH_WIN) or incompatible signal.")
    elif totals == 0:
        print("HITS=0 → FOLD_MIN and/or MIN_AREA too high. Adjust using p95/p99 from the profile.")
    print("\nDone.")

# Run
main()


== 2) QC: open BigWigs and determine common chromosomes ==
[QC] Common chromosomes (canonical): ['NC_002505', 'NC_002506']
Checking inputs...
True /content/drive/MyDrive/colab_data/20250525_RNAseq_traces_stranded/CRP_A_plus_stranded.bw
True /content/drive/MyDrive/colab_data/20250525_RNAseq_traces_stranded/CRP_A_minus_stranded.bw
True /content/drive/MyDrive/colab_data/20250525_RNAseq_traces_stranded/K52Q_A_plus_stranded.bw
True /content/drive/MyDrive/colab_data/20250525_RNAseq_traces_stranded/K52Q_A_minus_stranded.bw
True /content/drive/MyDrive/colab_data/20250821_chip_analysis/data/raw/20250220_NC002505_GFF3converted.xlsx
True /content/drive/MyDrive/colab_data/20250821_chip_analysis/data/raw/20250220_NC002506_GFF3converted.xlsx
True /content/drive/MyDrive/colab_data/20250821_chip_analysis/data/raw/250221_TSS_plus.txt
True /content/drive/MyDrive/colab_data/20250821_chip_analysis/data/raw/250221_TSS_minus.txt

== 3) Healthy mask (CDS, rRNA, tRNA) and intergenics ==
       chrom   len_bp 